## Analysis
This project demonstrated the usage of pkdb for the test substance caffeine.
### Data Sources
- pkdb rest api : "http://0.0.0.0:8000/api/v1/"

### Outputs:
- data/processed/outputs.tsv
- data/processed/timecourses.tsv
- data/processed/interventions.tsv
- data/processed/individuals.tsv
- data/processed/groups.tsv

#### Groups and Individuals 
- data/processed/all_subjects.tsv

### Merged Dataframes
- data/processed/all_subjects.tsv

### Normalized Dataframes
- data/processed/all_subjects.tsv

### Changes
- 11-29-2018 : Started project

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import PkdbModel, Preprocessed
import pandas as pd

In [3]:
queries = [
    'outputs', 
    'timecourses',
    'interventions',
    'individuals',
    'groups',
    'studies',
    'substances'
]
#queries = ["studies","substances"]
#queries = ["substances"]
#queries = ["studies"]
#queries = ["individuals"]

### Query, Process and Save
Json data is loaded, transformed to dataframes, preprocessed and saved as tab seperated files.

In [4]:
for pk_instance in queries:
    pk_instance = PkdbModel(pk_instance)  
    pk_instance.load()
    pk_instance.preprocess()
    pk_instance.save()
    pk_instance.report()

____________________________________________________________
Name: outputs
Loaded: True
Preprocessed: True
saved: True
outputs were succsesfully saved to </home/mkoenig/git/pkdb_analysis/data/1-preprocessed/outputs.tsv>
____________________________________________________________
Name: timecourses
Loaded: True
Preprocessed: True
saved: True
timecourses were succsesfully saved to </home/mkoenig/git/pkdb_analysis/data/1-preprocessed/timecourses.tsv>
____________________________________________________________
Name: interventions
Loaded: True
Preprocessed: True
saved: True
interventions were succsesfully saved to </home/mkoenig/git/pkdb_analysis/data/1-preprocessed/interventions.tsv>
____________________________________________________________
Name: individuals
Loaded: True
Preprocessed: True
saved: True
individuals were succsesfully saved to </home/mkoenig/git/pkdb_analysis/data/1-preprocessed/individuals.tsv>
____________________________________________________________
Name: groups
Load

In [5]:
studies = PkdbModel('studies',destination='1-preprocessed')
studies.read()
frames = []
for number,row in studies.data.dropna(subset=["substances"]).iterrows():
    substances = [{"study":row["name"],"substance":x.strip()} for x in row["substances"].split(",")]
    frames.append(pd.DataFrame(substances))

study_substance_map = pd.concat(frames)

def agg_studies(x):
    return pd.Series([','.join(x["study"]),len(x["study"])], index = ["studies","study_number"])
    
substance_study = study_substance_map.groupby(['substance']).apply(agg_studies)
substance_study = substance_study.reset_index()

In [6]:
substances = PkdbModel('substances',destination='1-preprocessed')
substances.read()
substances.data = substance_study.merge(substances.data, left_on="substance", right_on="name")



substances.data.sort_values(by="study_number", ascending=False, inplace=True)
substances.save()

In [7]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["calculated"]).apply(len)

calculated
False    10430
True      6363
dtype: int64

In [8]:
this_data = PkdbModel('interventions',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["unit"]).apply(len)

unit
IU / hour / kilogram        1
g/kg                        6
gram                      538
gram / hour                12
gram / kilogram            65
gram / meter ** 2           1
mole                        1
mole / hour                 1
mole / hour / kilogram      4
mole / kilogram             1
yr                          1
dtype: int64

In [9]:
this_data = PkdbModel('outputs',destination='1-preprocessed')
this_data.read()
this_data.data.groupby(["measurement_type","unit"]).count()

allowed_users  \
measurement_type               unit                                                       
Glucose oral disposition index none                                                   6   
HOMA-IR                        none                                                   6   
Insulinogenic index            none                                                   6   
Matsuda index                  none                                                   6   
QUICKI                         none                                                   6   
age                            yr                                                     2   
amount                         gram                                                  96   
auc_end                        IU * hour / liter                                     12   
                               gram * hour / liter                                 1289   
                               hour * mole / liter                                   15   
auc_inf                        IU * hour / liter                                     30   
                               gram * hour / liter                                 1545   
                               hour * mole / liter                                   16   
auc_relative                   dimensionless                                         27   
aumc_inf                       gram * hour ** 2 / liter                               2   
bioavailability                dimensionless                                         13   
clearance                      gram * milliliter / IU / hour                          8   
                               gram * milliliter / IU / hour / kilogram               6   
                               liter / hour                                        1304   
                               liter / hour / kilogram                             1237   
                               microgram * milliliter / hour / micromole              5   
                               milliliter / hour / micromole                          2   
                               milliliter / meter ** 2 / minute                      79   
clearance_intrinsic            liter / hour                                          19   
                               liter / hour / kilogram                                5   
clearance_intrinsic_unbound    liter / hour / kilogram                                2   
clearance_partial              liter / hour                                          31   
                               liter / hour / kilogram                               51   
clearance_renal                liter / hour                                         216   
                               liter / hour / kilogram                               85   
...                                                                                 ...   
fraction_absorbed              none                                                   5   
fraction_unbound               dimensionless                                         26   
kabs                           1 / minute                                            28   
kel                            1 / minute                                          1000   
lag_absorption                 hour                                                   4   
metabolic ratio                dimensionless                                         46   
                               none                                                 720   
mrt                            hour                                                  62   
oro-cecal transit time         hour                                                  12   
plasma_binding                 dimensionless                                         10   
                               none                                                   2   
ra                             1 / minute                             

### Merge
 - merge individuals and groups  --> all_subjects,
 - merge outputs and timecourses --> all_results, 
 - merge all_results with interventions and individuals  --> individuals_complete, 
 - merge all_results with interventions and groups  --> groups_complete, 
 - merge all_results with interventions and all_subjects -->  all_complete

In [10]:
prepocessed = Preprocessed()

In [11]:
prepocessed.read()
prepocessed.merge()
prepocessed.save()

/home/mkoenig/envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/mkoenig/envs/pkdb_analysis/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
